In [3]:
import numpy as np

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sympy as sym
from sympy.plotting import plot3d


from ipywidgets import interact
from ipywidgets import FloatSlider

from IPython.display import clear_output

x, y = sym.symbols('x y')
%matplotlib widget
# import matplotlib
# matplotlib.rcParams['text.usetex'] = True
plt.ion()

# 2D Points
**2D points.** (pixel coordinates in an image) can be denoted using a pair of values $\textbf{x}=(x,y) \in \mathcal{R}^2$ or alternatively,
$$
\textbf{x} = \begin{bmatrix}
x \\ 
y
\end{bmatrix}
$$
(As stated in the introduction, we use the $(x_1, x_2, \dots)$ notation to denote column vectors.)

In [4]:
class Point2Dplot:
    def __init__(self, im_plane_size = (10, 10)):
        x_size, y_size = im_plane_size
        x_size_h, y_size_h = x_size / 2, y_size / 2
        fig, self.ax = plt.subplots()
        self.setup(x_size_h, y_size_h)
        
        self.sc = self.ax.scatter([1], [1], label = 'Point Location', color='blue')
        leg = self.ax.legend(loc='upper right')
        plt.tight_layout()

        x_slider=FloatSlider(min=-x_size_h, max=x_size_h, value=0)
        y_slider=FloatSlider(min=-y_size_h, max=y_size_h, value=0)
        
        interact(self.update, x_pos=x_slider, y_pos=y_slider)
        
    def setup(self, x_size_h, y_size_h):
        self.ax.set_xlim([-x_size_h, x_size_h])
        self.ax.set_ylim([-y_size_h, y_size_h])


        self.ax.set_title("Point on 2D image plane")
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('y')
        self.ax.grid(True)

    def update(self, x_pos, y_pos):
        self.sc.remove()
        self.sc = self.ax.scatter(x_pos, y_pos, color='blue')
        
p2p = Point2Dplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='x_pos', max=5.0, min=-5.0), FloatSlider(value=0.0, d…

## Homogeneous coordinates

In mathematics, **homogeneous coordinates** or **projective coordinates**, introduced by [August Ferdinand Möbius](https://en.wikipedia.org/wiki/August_Ferdinand_M%C3%B6bius) in his 1827 work Der barycentrische Calcul, are a system of coordinates used in [projective geometry](https://en.wikipedia.org/wiki/Projective_geometry), as [Cartesian coordinates](https://en.wikipedia.org/wiki/Cartesian_coordinate_system) are used in [Euclidean geometry](https://en.wikipedia.org/wiki/Euclidean_geometry). 

2D points can also be represented using *homogeneueous coordinates*, $\tilde{\textbf{x}}=(\tilde{x}, \tilde{y}, \tilde{w}) \in \mathcal{P}^2$, where vectors that differ only by scale are considered to be equivalent. $\mathcal{P}^2 = \mathcal{R}^3 - (0, 0, 0)$ is called 2D *projective space*.

A homogeneous vector $\tilde{\textbf{x}}$ can be converted back into an *inhomogeneous* vector $\textbf{x}$ by dividing through by the last element $\tilde{w}$ i.e.,

$$ \tilde{\textbf{x}} = (\tilde{x}, \tilde{y}, \tilde{w}) = \tilde{w} (x, y, 1) = \tilde{w}\bar{\textbf{x}}$$

where $\bar{\textbf{x}} = (x, y, 1)$ is and *augumented vector*. Homogeneous points whose last element is $\tilde{w} = 0$ is called *ideal points or points at infinity* and do not have an equivalent inhomogeneous representation. 

In [33]:
class Point2Dray3D:
    def __init__(self, im_plane_size = (6, 6), depth=5):
        x_size, y_size = im_plane_size
        x_size_h, y_size_h = x_size / 2, y_size / 2
        view_point_x = x_size_h * depth
        view_point_y = y_size_h * depth
        fig = plt.figure()
        self.ax = fig.add_subplot(projection='3d')

        self.ax.set_xlim([-view_point_x, view_point_x])
        self.ax.set_ylim([0, depth])
        self.ax.set_zlim([-view_point_y, view_point_y])

        self.ax.set_title("3D point projection on Image plane")
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('z')
        self.ax.set_zlabel('y')
        
        self.center = np.zeros(3)
        self.pov = self.ax.scatter(*self.center, color='black')

        xx, yy = np.meshgrid(np.arange(-x_size_h, x_size_h+1), np.arange(-y_size_h, y_size_h+1))
        zz = np.ones_like(xx)

        self.im_plane = self.ax.plot_surface(xx, zz, yy, alpha=0.3, color="blue")
        self.ax.text(-x_size_h, 1, y_size_h, "Image Plane", fontsize=6, color='blue')
        
        self.ax.plot(*tuple(zip(self.center, [-view_point_x, depth, view_point_y])), color='grey', alpha=0.5)
        self.ax.plot(*tuple(zip(self.center, [view_point_x, depth, view_point_y])), color='grey', alpha=0.5)
        self.ax.plot(*tuple(zip(self.center, [view_point_x, depth, -view_point_y])), color='grey', alpha=0.5)
        self.ax.plot(*tuple(zip(self.center, [-view_point_x, depth, -view_point_y])), color='grey', alpha=0.5)
        
        
        vect = np.array([0, 1, 0])

        self.ray = self.ax.quiver(*self.center, *vect, color='green')
        self.ray_text = self.ax.text(*vect, str(vect), fontsize=6, color='green')
        self.im_point = self.ax.scatter(*vect, color='red')
        
        self.ax.legend([self.pov, self.ray, self.im_point], [r'Point of View: $x=(0, 0, 0)$', r'View Ray: Vect($\tilde{w}(x, y, 1)$)', 'Image point: $(x, y, 1)$'], loc='upper right',
                      shadow=True)
        plt.tight_layout()
        
        x_slider=FloatSlider(min=-x_size_h, max=x_size_h, value=0)
        y_slider=FloatSlider(min=-y_size_h, max=y_size_h, value=0)
        dist_slider=FloatSlider(min=0, max=depth, value=1)
        
        interact(self.update, x_pos=x_slider, y_pos=y_slider, dist=dist_slider)
        
    def update(self, x_pos, y_pos, dist):
        vect = np.array([x_pos, 1, y_pos])
        vect_scaled = vect * dist

        self.ray.remove()
        self.ray = self.ax.quiver(*self.center, *vect_scaled, color='green')
        self.ray_text.remove()
        self.ray_text = self.ax.text(*vect_scaled, str(vect_scaled), fontsize=6, color='green')

        self.im_point.remove()
        self.im_point = self.ax.scatter(*vect, color='red')
            
p2r3 = Point2Dray3D()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='x_pos', max=3.0, min=-3.0), FloatSlider(value=0.0, d…

#### Point Translation

In [25]:
T = np.array([
    [1, 0, 3],
    [0, 1, 6],
    [0, 0, 1]
])

point1 = np.array([3, 2, 1])

print((T @ (point1 * 5)) / 5, T @ point1)

[6. 8. 1.] [6 8 1]
